In [ ]:
%pip install cryptography

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import Encoding, ParameterFormat, PublicFormat
from cryptography.hazmat.primitives.serialization import load_pem_parameters, load_pem_public_key
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

In [3]:
with open ("shared_file", "rb") as file:
    parameters = load_pem_parameters(file.read()) 

In [4]:
private_key_D = parameters.generate_private_key() # Blue
public_mixed_with_D = private_key_D.public_key()  # light-blue-green

with open("shared_file", "wb") as file:
    file.write(public_mixed_with_D.public_bytes(encoding=Encoding.PEM, format=PublicFormat.SubjectPublicKeyInfo))

In [5]:
with open ("shared_file", "rb") as file:
    public_mixed_with_C = load_pem_public_key(file.read()) 

In [6]:
shared_key_C_then_D = private_key_D.exchange(public_mixed_with_C) # brown

derived_key_C_then_D = HKDF(
    algorithm = hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
    backend=default_backend()
).derive(shared_key_C_then_D)

print(derived_key_C_then_D)

b'\xdfm\xc0\x9d\x03\xbc\xa7\xb4\xaeU\xf2\xf0\x1eQ\x8d}ty\xc4\xd9\xc5\xaf\x1a\xd9\x10JW\t=:\x9e\x1d'


### Decryption without Integrity and Authenticity

In [23]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
from io import StringIO
import pandas as pd

In [24]:
with open ("shared_file", "rb") as file:
    columns_desc_enc = file.read()

In [ ]:
def decrypt_csv_ctr(ct, key, counter_iv, counter_size=128):
    counter = Counter.new(counter_size, initial_value=counter_iv)    
    aes = AES.new(key=key, mode=AES.MODE_CTR, counter=counter)
    return aes.decrypt(ct)

pd.read_csv(StringIO(decrypt_csv_ctr(columns_desc_enc, derived_key_C_then_D, 10).decode('utf-8')))

In [ ]:
with open ("shared_file", "rb") as file:
    infringement_dataset_enc = file.read()

pd.read_csv(StringIO(decrypt_csv_ctr(infringement_dataset_enc, derived_key_C_then_D, 10).decode('utf-8')))

### Decryption with Integrity and Authenticity

In [33]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from io import StringIO
import pandas as pd

In [34]:
with open ("shared_file", "rb") as file:
    columns_desc_enc = file.read()

In [ ]:
with open ("shared_file", "rb") as file:
    nonce = file.read()

aesgcm = AESGCM(derived_key_C_then_D)
pd.read_csv(StringIO(aesgcm.decrypt(nonce, columns_desc_enc, None).decode("utf-8")))

In [ ]:
with open ("shared_file", "rb") as file:
    infringement_dataset_enc = file.read()

In [ ]:
with open ("shared_file", "rb") as file:
    nonce = file.read()

pd.read_csv(StringIO(aesgcm.decrypt(nonce, infringement_dataset_enc, None).decode("utf-8")))